# Team Rating Model 

### Data Cleaning/Preprocessing

In [78]:
import pandas as pd
import numpy as np
import sklearn
import math

In [79]:
#prep the dataset with only the features we want
df=pd.read_csv('Data/Data.csv')
columns=df.columns
print(columns)
wanted_columns=['Player','Tm','ORtg','DRtg']
for column in wanted_columns:
    columns=columns.drop(column)
print(columns)
df=df.drop(columns=columns)
df

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TMP', 'ORtg',
       'DRtg', 'PER', 'TS%', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%',
       'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM',
       'BPM', 'VORP', 'PG%', 'SG%', 'SF%', 'PF%', 'C%', 'OnCourt', 'On-Off',
       'BadPass', 'LostBall', 'ShootFC', 'OffFC', 'ShootFD', 'OffFD', 'PGA',
       'And1', 'Blkd', 'Dist.', '%2PFGA', '0-3%FGA', '3-10%FGA', '10-16%FGA',
       '16-3P%FGA', '%3PFGA', '0-3FG%', '3-10FG%', '10-16FG%', '16-3PFG%',
       '2P%A', '3P%A', 'Dunk%FGA', 'Dunks', 'C3%3PA', 'C33P%', 'Season '],
      dtype='object')
Index(['Pos', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%',
       '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'TMP', 'P

,Player,Tm,ORtg,DRtg
0,Precious Achiuwa,TOR,112.0,113.0
1,Steven Adams,MEM,118.0,108.0
2,Bam Adebayo,MIA,115.0,111.0
3,Ochai Agbaji,UTA,115.0,121.0
4,Santi Aldama,MEM,120.0,111.0
...,...,...,...,...
675,Trae Young,ATL,116.0,119.0
676,Omer Yurtseven,MIA,126.0,112.0
677,Cody Zeller,MIA,124.0,114.0
678,Ivica Zubac,LAC,125.0,112.0


In [80]:
#Create a new column Ofensive rating - defensive rating 
#def NetRating(offrt,defrt):
#    return offrt-defrt
df['NetRtg']=df['ORtg']-df['DRtg']
df=df.drop(columns=['ORtg','DRtg'])
df=df.dropna()
df

,Player,Tm,NetRtg
0,Precious Achiuwa,TOR,-1.0
1,Steven Adams,MEM,10.0
2,Bam Adebayo,MIA,4.0
3,Ochai Agbaji,UTA,-6.0
4,Santi Aldama,MEM,9.0
...,...,...,...
674,Thaddeus Young,TOR,5.0
675,Trae Young,ATL,-3.0
676,Omer Yurtseven,MIA,14.0
677,Cody Zeller,MIA,10.0


In [81]:
list_of_teams=df['Tm'].tolist()
list_of_teams=set(list_of_teams)
list_of_teams=list(list_of_teams)
team_dict={}
for team in list_of_teams:
    df_temp=df[df['Tm']==team]
    df_temp=df_temp.drop(columns=['Tm'])
    team_dict[team]=df_temp.values.tolist()
def getTopN(team_roster,n):
    #this will be inputted as a list of lists with the first index being player and last index being rating
    #we want to get the the top n players from this 
    #I could create a data type for player and rating and sort based on that 
    #I could find the index of the largest and then remove it and add to anther list n times
    top_players=[]
    for i in range(n):
        max_rating=['',float('-inf')]
        max_index=0
        for i in range(len(team_roster)):
            player=team_roster[i]
            if player[1]>max_rating[1]:
                max_rating=player
                max_index=i
        team_roster.remove(max_rating)
        top_players.append(max_rating)
    return top_players
for key,value in team_dict.items():
    team_dict[key]=getTopN(value,5)
for key,value in team_dict.items():
    print(key,value)

MIL [['Jae Crowder', 22.0], ['Brook Lopez', 13.0], ['Meyers Leonard', 10.0], ['Grayson Allen', 9.0], ['Giannis Antetokounmpo', 9.0]]
OKC [['Shai Gilgeous-Alexander', 12.0], ['Kenrich Williams', 12.0], ['Mike Muscala', 11.0], ['Isaiah Joe', 9.0], ['Jeremiah Robinson-Earl', 6.0]]
PHO [['Kevin Durant', 13.0], ['Cameron Johnson', 13.0], ['Chris Paul', 10.0], ['Deandre Ayton', 8.0], ['Jock Landale', 6.0]]
MEM [['Luke Kennard', 31.0], ['Brandon Clarke', 22.0], ['Xavier Tillman Sr.', 20.0], ['Jaren Jackson Jr.', 13.0], ['Steven Adams', 10.0]]
WAS [['Jordan Schakel', 55.0], ['Jay Huff', 35.0], ['Daniel Gafford', 22.0], ['Delon Wright', 15.0], ['Anthony Gill', 8.0]]
PHI [['Paul Reed', 17.0], ['Joel Embiid', 15.0], ['Montrezl Harrell', 11.0], ['James Harden', 10.0], ['Louis King', 9.0]]
CHI [['Terry Taylor', 54.0], ['Tony Bradley', 25.0], ['Javonte Green', 13.0], ['Andre Drummond', 10.0], ['Derrick Jones Jr.', 10.0]]
BOS [['Robert Williams', 36.0], ['Luke Kornet', 32.0], ['Al Horford', 21.0], ['

In [82]:
df_team=pd.read_csv('Data/TeamData.csv')
#df_team
#converting team name so it lines up
#when player data is in organize it into a dataframe for the 
df_players=pd.read_csv('Data/AllPlayerNoSalaryCleaned.csv')
df_players


,Unnamed: 0,Player,Tm,G,3P,3P%,2P,2P%,FT%,ORB,...,3-10FG%,10-16FG%,16-3PFG%,2P%A,3P%A,Dunk%FGA,C3%3PA,C33P%,Season,TruePos
0,0,Precious Achiuwa,TOR,55.0,0.5,0.269,3.0,0.564,0.702,1.8,...,0.417,0.276,0.200,0.629,1.000,0.141,0.444,0.250,2023.0,4.83
1,1,Steven Adams,MEM,42.0,0.0,0.000,3.7,0.599,0.364,5.1,...,0.500,0.400,NaN,0.503,NaN,0.163,1.000,0.000,2023.0,5.00
2,2,Bam Adebayo,MIA,75.0,0.0,0.083,8.0,0.545,0.806,2.5,...,0.478,0.498,0.288,0.601,1.000,0.136,0.000,NaN,2023.0,5.00
3,3,Ochai Agbaji,UTA,59.0,1.4,0.355,1.4,0.532,0.812,0.7,...,0.494,0.400,0.444,0.762,0.988,0.054,0.412,0.457,2023.0,1.90
4,4,Santi Aldama,MEM,77.0,1.2,0.353,2.0,0.591,0.750,1.1,...,0.471,0.250,0.444,0.680,0.968,0.118,0.391,0.365,2023.0,4.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10381,10381,Dedric Willoughby,CHI,25.0,1.2,0.296,1.3,0.395,0.765,0.4,...,0.214,0.300,0.447,0.688,0.759,0.000,0.163,0.313,NaN,1.33
10382,10382,Haywoode Workman,TOT,36.0,0.4,0.326,0.5,0.362,0.667,0.0,...,0.000,0.286,0.391,0.529,0.786,0.000,0.349,0.400,NaN,1.11
10383,10383,Metta World Peace,CHI,72.0,0.8,0.314,3.5,0.438,0.674,0.9,...,0.268,0.242,0.361,0.518,0.917,0.040,0.283,0.278,NaN,2.74
10384,10384,Lorenzen Wright,ATL,75.0,0.0,0.333,2.4,0.500,0.644,1.6,...,0.424,0.385,0.353,0.536,1.000,0.089,0.333,1.000,NaN,4.71


In [87]:
columns=df_players.columns
print(columns)
wanted_columns=['Player','Tm','ORtg','DRtg','Season']
for column in wanted_columns:
    columns=columns.drop(column)
df_player=df_players.drop(columns=columns)
df_player

Index(['Unnamed: 0', 'Player', 'Tm', 'G', '3P', '3P%', '2P', '2P%', 'FT%',
       'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'ORtg', 'DRtg', 'ORB%',
       'DRB%', 'AST%', 'STL%', 'BLK%', 'USG%', 'PG%', 'SG%', 'SF%', 'PF%',
       'C%', 'PGA', 'Blkd', 'Dist.', '%2PFGA', '0-3%FGA', '3-10%FGA',
       '10-16%FGA', '16-3P%FGA', '%3PFGA', '0-3FG%', '3-10FG%', '10-16FG%',
       '16-3PFG%', '2P%A', '3P%A', 'Dunk%FGA', 'C3%3PA', 'C33P%', 'Season',
       'TruePos'],
      dtype='object')


,Player,Tm,ORtg,DRtg,Season
0,Precious Achiuwa,TOR,112.0,113.0,2023.0
1,Steven Adams,MEM,118.0,108.0,2023.0
2,Bam Adebayo,MIA,115.0,111.0,2023.0
3,Ochai Agbaji,UTA,115.0,121.0,2023.0
4,Santi Aldama,MEM,120.0,111.0,2023.0
...,...,...,...,...,...
10381,Dedric Willoughby,CHI,96.0,106.0,NaN
10382,Haywoode Workman,TOT,99.0,108.0,NaN
10383,Metta World Peace,CHI,95.0,103.0,NaN
10384,Lorenzen Wright,ATL,106.0,106.0,NaN


In [88]:
#get this data into a df with just netrating and team X y training set for the model
#i need to watch the encoder video 
df_player.dropna(subset=['Season'], inplace=True)
df_player

,Player,Tm,ORtg,DRtg,Season
0,Precious Achiuwa,TOR,112.0,113.0,2023.0
1,Steven Adams,MEM,118.0,108.0,2023.0
2,Bam Adebayo,MIA,115.0,111.0,2023.0
3,Ochai Agbaji,UTA,115.0,121.0,2023.0
4,Santi Aldama,MEM,120.0,111.0,2023.0
...,...,...,...,...,...
2350,Thaddeus Young,IND,113.0,106.0,2019.0
2351,Trae Young,ATL,107.0,117.0,2019.0
2352,Cody Zeller,CHO,123.0,111.0,2019.0
2353,Ante Žižić,CLE,116.0,117.0,2019.0


In [89]:
df_player.head


<bound method NDFrame.head of                 Player   Tm   ORtg   DRtg  Season
0     Precious Achiuwa  TOR  112.0  113.0  2023.0
1         Steven Adams  MEM  118.0  108.0  2023.0
2          Bam Adebayo  MIA  115.0  111.0  2023.0
3         Ochai Agbaji  UTA  115.0  121.0  2023.0
4         Santi Aldama  MEM  120.0  111.0  2023.0
...                ...  ...    ...    ...     ...
2350    Thaddeus Young  IND  113.0  106.0  2019.0
2351        Trae Young  ATL  107.0  117.0  2019.0
2352       Cody Zeller  CHO  123.0  111.0  2019.0
2353        Ante Žižić  CLE  116.0  117.0  2019.0
2354       Ivica Zubac  TOT  116.0  108.0  2019.0

[2355 rows x 5 columns]>

In [96]:
# we will need to make and append to a dataframe and the dataframe should look like 
# player 1 -through- player n, team name, year, team rating
years=range(2000,2024)
for year in years:
    temp_df=df_player[df_player['Season']==year]
    #print(temp_df)
    

In [97]:
df_team
dict_team_names={'Boston Celtics':'BOS','Cleveland Cavaliers':'CLE','Philadelphia 76ers':'PHI','Memphis Grizzlies':'MEM','Milwaukee Bucks':'MIL','Vancouver Grizzlies':'VAN','Atlanta Hawks':'ATL','Golden State Warriors':'GSW','Chicago Bulls':'CHI','Los Angeles Clippers':'LAC'}
print(len(dict_team_names))

10
